## Kombinasi 3 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier capping with winsorization
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [161]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [162]:
df = pd.read_csv("../UFC_train.csv")

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [163]:
# Delete duplicate
print(f"Shape before dropping duplicate : {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate : {df.shape}")

Shape before dropping duplicate : (5410, 144)
Shape after dropping duplicate : (5410, 144)


In [164]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [165]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [166]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [167]:
# Impute missing value using iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(random_state=42)
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)

/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [168]:
# Drop R_fighter, B_fighter, Referee, location, date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [169]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.186013,0.182146,0.451630,0.434716,0.325559,0.299475,...,0.0,0.0,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.211328,0.193034,0.451822,0.445884,0.299806,0.279493,...,0.0,0.0,0.0,0.0,0.0,185.42,189.230788,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,1.0,5.0,6.0,3.0,0.0,177.80,185.420000,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,1.0,160.02,167.640000,135.0,34.0,31.0


In [170]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
B_Stance     64  0.011830


In [171]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

In [172]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [173]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [174]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [175]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 532
B_avg_opp_KD             461
B_avg_SIG_STR_pct        448
B_avg_opp_SIG_STR_pct    425
B_avg_TD_pct             186
                        ... 
R_Height_cms              19
R_Reach_cms               81
R_Weight_lbs             301
B_age                     31
R_age                     75
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 9.833641404805915%
B_avg_opp_KD = 8.521256931608132%
B_avg_SIG_STR_pct = 8.280961182994455%
B_avg_opp_SIG_STR_pct = 7.8558225508317925%
B_avg_TD_pct = 3.4380776340110906%
B_avg_opp_TD_pct = 5.619223659889094%
B_avg_SUB_ATT = 5.508317929759704%
B_avg_opp_SUB_ATT = 5.878003696857672%
B_avg_REV = 6.913123844731977%
B_avg_opp_REV = 11.090573012939002%
B_avg_SIG_STR_att = 6.358595194085027%
B_avg_SIG_STR_landed = 5.582255083179297%
B_avg_opp_SIG_STR_att = 6.22920517560074%
B_avg_opp_SIG_STR_landed = 6.598890942698706%
B_avg_TOTAL_STR_att = 4.787430683918669%
B_avg_TOTAL_STR_

In [176]:
from scipy.stats.mstats import winsorize

from scipy.stats.mstats import winsorize

# Identify columns with more than 0.5% outliers
outlier_columns = []
for key in outliers.keys():
    if outliers[key] / df_number[key].shape[0] > 0.005:
        outlier_columns.append(key)

# Winsorize the identified columns
for column in outlier_columns:
    df_number[column] = winsorize(df_number[column], limits=(0, 0.1))

# Combine df_object and df_number
df = pd.concat([df_object, df_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.186013,0.182146,0.451630,0.434716,0.325559,0.299475,...,0.0,0.0,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.211328,0.193034,0.451822,0.445884,0.299806,0.279493,...,0.0,0.0,0.0,0.0,0.0,185.42,189.230788,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.649799,0.000000,0.587031,0.185547,0.088281,0.104375,...,1.0,4.0,4.0,2.0,0.0,177.80,185.420000,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,0.0,160.02,167.640000,135.0,34.0,31.0


In [177]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                   0
B_avg_opp_KD               1
B_avg_SIG_STR_pct        202
B_avg_opp_SIG_STR_pct    158
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               35
R_Weight_lbs               0
B_age                      0
R_age                      4
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 0.0%
B_avg_opp_KD = 0.018484288354898334%
B_avg_SIG_STR_pct = 3.7338262476894637%
B_avg_opp_SIG_STR_pct = 2.9205175600739373%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 0.0%
B_avg_opp_SUB_ATT = 0.0%
B_avg_REV = 0.018484288354898334%
B_avg_opp_REV = 11.090573012939002%
B_avg_SIG_STR_att = 0.0%
B_avg_SIG_STR_landed = 0.0%
B_avg_opp_SIG_STR_att = 0.0%
B_avg_opp_SIG_STR_landed = 0.0%
B_avg_TOTAL_STR_att = 0.0%
B_avg_TOTAL_STR_landed = 0.0%
B_avg_opp_TOTAL_STR_att = 0.0%
B_avg_opp_TOTAL_STR_landed = 0.0%
B_avg_TD_att = 0.0%
B_avg_TD_landed = 0.0%
B_avg_opp_T

In [178]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

/var/folders/53/kv053qr13vxg2_y_1lg6hjg80000gn/T/ipykernel_65704/187983017.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')


In [179]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Middleweight,Orthodox,Southpaw,Red,0.186013,0.182146,0.451630,0.434716,0.325559,0.299475,...,0.0,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0,male
1,Lightweight,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0,male
2,Welterweight,Orthodox,Orthodox,Blue,0.211328,0.193034,0.451822,0.445884,0.299806,0.279493,...,0.0,0.0,0.0,0.0,185.42,189.230788,170.0,28.0,31.0,male
3,Welterweight,Southpaw,Orthodox,Blue,0.649799,0.000000,0.587031,0.185547,0.088281,0.104375,...,4.0,4.0,2.0,0.0,177.80,185.420000,170.0,28.0,34.0,male
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,0.0,160.02,167.640000,135.0,34.0,31.0,male


In [180]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [181]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [182]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [183]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [184]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)

In [185]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,7,Orthodox,Southpaw,Red,0.186013,0.182146,0.451630,0.434716,0.325559,0.299475,...,0.0,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0,1
1,5,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0,1
2,6,Orthodox,Orthodox,Blue,0.211328,0.193034,0.451822,0.445884,0.299806,0.279493,...,0.0,0.0,0.0,0.0,185.42,189.230788,170.0,28.0,31.0,1
3,6,Southpaw,Orthodox,Blue,0.649799,0.000000,0.587031,0.185547,0.088281,0.104375,...,4.0,4.0,2.0,0.0,177.80,185.420000,170.0,28.0,34.0,1
4,3,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,0.0,160.02,167.640000,135.0,34.0,31.0,1


In [186]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
0,7,0.186013,0.182146,0.451630,0.434716,0.325559,0.299475,0.563510,0.437996,0.200562,...,0,0,0,0,0,1,0,0,0,1
1,5,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,0.156250,0.132812,0.000000,...,0,0,0,1,0,0,0,1,0,0
2,6,0.211328,0.193034,0.451822,0.445884,0.299806,0.279493,0.508395,0.472152,0.182111,...,0,0,0,1,0,0,0,1,0,0
3,6,0.649799,0.000000,0.587031,0.185547,0.088281,0.104375,0.093750,0.062500,0.000000,...,1,0,0,1,0,0,0,1,0,0
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.062500,0.000244,0.125000,...,0,0,0,1,0,0,0,0,0,1


In [195]:
# Export to CSV
df.to_csv('UFC_kombinasi3_wt_featureselection.csv', index=False)

In [188]:
from sklearn.tree import DecisionTreeRegressor

# Feature selection with Decision Tree
X = df.drop('B_Reach_cms', axis=1)
y = df['B_Reach_cms']

# Splitting data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor()

In [189]:
feature_importances = model.feature_importances_

In [190]:
top_n = 20
top_features_indices = feature_importances.argsort()[-top_n:][::-1]

In [191]:
top_features_indices

array([ 64,  65,   9, 134,  31, 132, 130,   0,  22,  86,   1,  62, 113,
        17,  18,  73,  39,  19,  47,  81])

In [192]:
# Get the names of the top features
top_features = X.columns[top_features_indices]
top_features

Index(['B_Height_cms', 'B_Weight_lbs', 'B_avg_REV', 'gender', 'B_avg_LEG_att',
       'B_age', 'R_Reach_cms', 'weight_class', 'B_avg_opp_TD_landed',
       'R_avg_opp_TD_att', 'B_avg_KD', 'B_win_by_Submission',
       'R_avg_opp_CTRL_time(seconds)', 'B_avg_opp_TOTAL_STR_att',
       'B_avg_opp_TOTAL_STR_landed', 'R_avg_opp_SUB_ATT', 'B_avg_CLINCH_att',
       'B_avg_TD_att', 'B_avg_CTRL_time(seconds)', 'R_avg_TOTAL_STR_landed'],
      dtype='object')

In [193]:
# Create a dataframe with only the top features
df_top_features = df[top_features]
df_top_features.head()


,B_Height_cms,B_Weight_lbs,B_avg_REV,gender,B_avg_LEG_att,B_age,R_Reach_cms,weight_class,B_avg_opp_TD_landed,R_avg_opp_TD_att,B_avg_KD,B_win_by_Submission,R_avg_opp_CTRL_time(seconds),B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,R_avg_opp_SUB_ATT,B_avg_CLINCH_att,B_avg_TD_att,B_avg_CTRL_time(seconds),R_avg_TOTAL_STR_landed
0,182.88,185.0,0.200562,1,6.573702,26.0,177.800000,7,1.056337,2.990673,0.186013,0.0,119.118031,92.655464,48.452845,0.443638,8.063378,2.413763,129.196369,53.547907
1,177.80,155.0,0.000000,1,6.523438,27.0,187.960000,5,0.085938,5.812500,0.649799,0.0,115.000000,91.882812,33.078125,0.312500,5.671875,2.000000,35.945312,87.593750
2,182.88,170.0,0.182111,1,6.573702,28.0,189.230788,6,1.121961,2.820735,0.211328,0.0,132.749212,92.789902,48.503628,0.420586,8.063378,2.587360,141.719812,53.930300
3,175.26,185.0,0.000000,1,3.421875,28.0,185.420000,6,0.445312,3.660690,0.649799,0.0,116.292816,52.046875,28.671875,0.007238,11.867188,2.578125,68.281250,41.379938
4,165.10,135.0,0.125000,1,8.278564,34.0,167.640000,3,1.380859,6.000000,0.500000,2.0,162.796875,125.859863,71.359619,0.000000,9.979004,5.937500,209.931152,72.429688


In [194]:
# Export to CSV
df_top_features.to_csv('UFC_kombinasi3.csv', index=False)
